In [120]:
import sys
# setting path
sys.path.append('../')

import pandas as pd

In [121]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
RESULTS_PATH = MAIN_PATH + "results/"

# COMPARATIVA GLOBAL

In [122]:
path = RESULTS_PATH + "results_camelyon.csv"
df_cam = pd.read_csv(path)
df_cam = df_cam.drop(columns=[c for c in df_cam.columns if "loss" in c])
path = RESULTS_PATH + "results_wssb.csv"
df_wssb = pd.read_csv(path)
df_wssb= df_wssb.drop(columns=[c for c in df_wssb.columns if ("loss" in c) or ("rec" in c)])
new_df_wssb = df_wssb[["patch_size", "pretraining_epochs", "n_samples", "theta_val", "sigmaRui_sq"]]
new_df_wssb["wssb_mean_test_psnr_gt_h"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_h" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_psnr_gt_e"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_e" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_psnr_gt"] = (new_df_wssb["wssb_mean_test_psnr_gt_h"] + new_df_wssb["wssb_mean_test_psnr_gt_e"])/2.0
new_df_wssb["wssb_mean_test_ssim_gt_h"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_h" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt_e"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_e" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt"] = (new_df_wssb["wssb_mean_test_ssim_gt_h"] + new_df_wssb["wssb_mean_test_ssim_gt_e"])/2.0
new_df_wssb

<ipython-input-122-28509cd8626e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_wssb["wssb_mean_test_ssim_gt_h"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_h" in c]].mean(axis=1)
<ipython-input-122-28509cd8626e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_wssb["wssb_mean_test_ssim_gt_e"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_e" in c]].mean(axis=1)
<ipython-input-122-28509cd8626e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,patch_size,pretraining_epochs,n_samples,theta_val,sigmaRui_sq,wssb_mean_test_ssim_gt_h,wssb_mean_test_ssim_gt_e,wssb_mean_test_ssim_gt,wssb_mean_test_psnr_gt_h,wssb_mean_test_psnr_gt_e,wssb_mean_test_psnr_gt
0,224,0,60000,0.00,0.05,0.724696,0.927196,0.825946,19.606989,9.384888,14.495938
1,224,0,60000,0.50,0.05,0.922209,0.937829,0.930019,23.538993,18.362077,20.950535
2,224,0,60000,0.75,0.05,0.766481,0.918176,0.842329,15.727535,-15.297075,0.215230
3,224,0,60000,0.90,0.05,0.413317,0.918209,0.665763,-13.399599,-8.569815,-10.984707
4,224,0,60000,0.99,0.05,0.413698,0.918203,0.665951,-18.640905,-10.975104,-14.808005


In [127]:
df = pd.merge(df_cam, new_df_wssb, on=["patch_size", "n_samples", "theta_val", "sigmaRui_sq", "pretraining_epochs"])
df = df.drop(columns=["patch_size", "n_samples", "sigmaRui_sq"] + [c for c in df.columns if ("_h" in c) or ("_e" in c)])
df.columns = ["$\\theta$", "Rec. MSE (Camelyon)", "Rec. PSNR (Camelyon)", "Rec. SSIM (Camelyon)", "GT PSNR (WSSB)", "GT SSIM (WSSB)"]
df

,$\theta$,Rec. MSE (Camelyon),Rec. PSNR (Camelyon),Rec. SSIM (Camelyon),GT PSNR (WSSB),GT SSIM (WSSB)
0,0.00,0.000736,22.552444,0.845705,0.825946,14.495938
1,0.50,0.000846,20.999090,0.966937,0.930019,20.950535
2,0.75,0.002057,17.634794,0.883150,0.842329,0.215230
3,0.90,0.023112,5.824536,0.259591,0.665763,-10.984707
4,0.99,0.023116,5.822108,0.259629,0.665951,-14.808005


In [129]:
styler = df.style
styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns))]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Summary of results.", label="tab:results_summary",
    )
print(s)

\begin{table}
\centering
\caption{Summary of results.}
\label{tab:results_summary}
\begin{tabular}{cccccc}
\toprule
$\theta$ & Rec. MSE (Camelyon) & Rec. PSNR (Camelyon) & Rec. SSIM (Camelyon) & GT PSNR (WSSB) & GT SSIM (WSSB) \\
\midrule
0.000000 & 0.000736 & 22.552444 & 0.845705 & 0.825946 & 14.495938 \\
0.500000 & 0.000846 & 20.999090 & 0.966937 & 0.930019 & 20.950535 \\
0.750000 & 0.002057 & 17.634794 & 0.883150 & 0.842329 & 0.215230 \\
0.900000 & 0.023112 & 5.824536 & 0.259591 & 0.665763 & -10.984707 \\
0.990000 & 0.023116 & 5.822108 & 0.259629 & 0.665951 & -14.808005 \\
\bottomrule
\end{tabular}
\end{table}



# CAMELYON

In [119]:
path = RESULTS_PATH + "results_camelyon.csv"
df = pd.read_csv(path)
df = df.drop(columns=["patch_size", "n_samples", "sigmaRui_sq"] + ["camelyon_test_loss", "camelyon_test_loss_mse", "camelyon_test_loss_kl"])

#df["Version"] = ["New" for i in range(len(df))]
#sh_row = {"Version" : "Old", "patch_size" : 64, "pretraining_epochs" : 1, "n_samples" : "?", "lambda_val" : "$\\theta=0.8$", "sigmaRui_sq" : "0.05", "camelyon_test_mse_rec" : "0.0712", "camelyon_test_ssim_rec" : "?", "camelyon_test_ssim_rec" : "?"}
#df = df.append(sh_row, ignore_index=True)
#df.columns = ["Patch size", "Pretraining epochs", "Num. samples", "$\theta$", "$\sigma_{Rui}^2$", "Reconstruction MSE", "Reconstruction PSNR", "Reconstruction SSIM", "Version"]
#df = df[["Version"] + list(df.columns[:-1])]
#df.columns = ["Patch size", "Pretraining epochs", "Num. samples", "$\\theta$", "$\sigma_{Rui}^2$", "Reconstruction MSE", "Reconstruction PSNR", "Reconstruction SSIM"]
df.columns = ["Pre. epochs", "$\\theta$", "Reconstruction MSE", "Reconstruction PSNR", "Reconstruction SSIM"]
df

,pretraining_epochs,$\theta$,Reconstruction MSE,Reconstruction PSNR,Reconstruction SSIM
0,0,0.00,0.000736,22.552444,0.845705
1,0,0.50,0.000846,20.999090,0.966937
2,0,0.75,0.002057,17.634794,0.883150
3,0,0.90,0.023112,5.824536,0.259591
4,0,0.99,0.023116,5.822108,0.259629


In [53]:
styler = df.style
styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns))]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Results on Camelyon test dataset.", label="tab:results_camelyon",
    )
print(s)
# Fix header

\begin{table}
\centering
\caption{Results on Camelyon test dataset.}
\label{tab:results_camelyon}
\begin{tabular}{cccccccc}
\toprule
Patch size & Pretraining epochs & Num. samples & $	heta$ & $\sigma_{Rui}^2$ & Reconstruction MSE & Reconstruction PSNR & Reconstruction SSIM \\
\midrule
224 & 0 & 60000 & 0.000000 & 0.050000 & 0.000736 & 22.552444 & 0.845705 \\
224 & 0 & 60000 & 0.500000 & 0.050000 & 0.000846 & 20.999090 & 0.966937 \\
224 & 0 & 60000 & 0.750000 & 0.050000 & 0.002057 & 17.634794 & 0.883150 \\
224 & 0 & 60000 & 0.900000 & 0.050000 & 0.023112 & 5.824536 & 0.259591 \\
224 & 0 & 60000 & 0.990000 & 0.050000 & 0.023116 & 5.822108 & 0.259629 \\
\bottomrule
\end{tabular}
\end{table}



# WSSB

In [64]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns=[c for c in df.columns if "loss" in c])
theta_list = list(df["theta_val"])
#df[[c for c in df.columns if "mse_rec" in c]]

df = df[df["pretraining_epochs"] == 0] 
df = df[[c for c in df.columns if "mse_rec" in c]]
df["wssb_mean_test_mse_rec"] = df.mean(axis=1)
df["Name"] = [f"BCDnet $\\theta={t}$" for t in theta_list]
df = df[["Name"] + list(df.columns[:-1])]

#df = df.append({"Name" : "BCDnet (old)", "wssb_Lung_test_mse_rec" : 0.0066, "wssb_Breast_test_mse_rec" : 0.0172, "wssb_Colon_test_mse_rec" : 0.0092, "wssb_mean_test_mse_rec" : 0.0110}, ignore_index=True)
df = df.append({"Name" : "BKSVD", "wssb_Lung_test_mse_rec" : 0.0004, "wssb_Breast_test_mse_rec" : 0.0003, "wssb_Colon_test_mse_rec" : 0.0003, "wssb_mean_test_mse_rec" : 0.0003}, ignore_index=True)
df

<ipython-input-64-dfa226347a17>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name" : "BKSVD", "wssb_Lung_test_mse_rec" : 0.0004, "wssb_Breast_test_mse_rec" : 0.0003, "wssb_Colon_test_mse_rec" : 0.0003, "wssb_mean_test_mse_rec" : 0.0003}, ignore_index=True)


,Name,wssb_Lung_test_mse_rec,wssb_Breast_test_mse_rec,wssb_Colon_test_mse_rec,wssb_mean_test_mse_rec
0,BCDnet $\theta=0.0$,0.000234,0.000859,0.000493,0.000529
1,BCDnet $\theta=0.5$,0.000057,0.000429,0.000034,0.000173
2,BCDnet $\theta=0.75$,0.000430,0.002035,0.000884,0.001116
3,BCDnet $\theta=0.9$,0.004327,0.014658,0.007764,0.008916
4,BCDnet $\theta=0.99$,0.004327,0.014658,0.007765,0.008917
5,BKSVD,0.000400,0.000300,0.000300,0.000300


In [65]:
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Breast", "Colon", "Mean"]
df = df[["Organ"] + list(df.columns[:-1])]
df

,Organ,BCDnet $\theta=0.0$,BCDnet $\theta=0.5$,BCDnet $\theta=0.75$,BCDnet $\theta=0.9$,BCDnet $\theta=0.99$,BKSVD
0,Lung,0.000234,0.000057,0.00043,0.004327,0.004327,0.0004
1,Breast,0.000859,0.000429,0.002035,0.014658,0.014658,0.0003
2,Colon,0.000493,0.000034,0.000884,0.007764,0.007765,0.0003
3,Mean,0.000529,0.000173,0.001116,0.008916,0.008917,0.0003


In [66]:
styler = df.style
styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns))]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Reconstruction MSE on WSSB dataset.", label="tab:mse_rec_wssb",
    )
print(s)

\begin{table}
\centering
\caption{Reconstruction MSE on WSSB dataset.}
\label{tab:mse_rec_wssb}
\begin{tabular}{ccccccc}
\toprule
Organ & BCDnet $\theta=0.0$ & BCDnet $\theta=0.5$ & BCDnet $\theta=0.75$ & BCDnet $\theta=0.9$ & BCDnet $\theta=0.99$ & BKSVD \\
\midrule
Lung & 0.000234 & 0.000057 & 0.000430 & 0.004327 & 0.004327 & 0.000400 \\
Breast & 0.000859 & 0.000429 & 0.002035 & 0.014658 & 0.014658 & 0.000300 \\
Colon & 0.000493 & 0.000034 & 0.000884 & 0.007764 & 0.007765 & 0.000300 \\
Mean & 0.000529 & 0.000173 & 0.001116 & 0.008916 & 0.008917 & 0.000300 \\
\bottomrule
\end{tabular}
\end{table}



## Stain metrics

In [57]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
theta_list = list(df["theta_val"])
df = df.drop(columns=[c for c in df.columns if "loss" in c])
df = df[df["pretraining_epochs"] == 0] 
df = df[[c for c in df.columns if "psnr_gt" in c]]
df["Name"] = [f"BCDnet $\\theta={t}$" for t in theta_list]
df = df[["Name"] + list(df.columns[:-1])]
#df = df.append({"Name" : "BCDnet (old)", 
#    "wssb_Lung_test_psnr_gt_h" : 27.06, "wssb_Lung_test_psnr_gt_e" : 25.69, 
#    "wssb_Breast_test_psnr_gt_h" : 21.81, "wssb_Breast_test_psnr_gt_e" : 24.53, 
#    "wssb_Colon_test_psnr_gt_h" : 20.32, "wssb_Colon_test_psnr_gt_e" : 21.70}, ignore_index=True)
df = df.append({"Name" : "BKSVD", 
    "wssb_Lung_test_psnr_gt_h" : 32.67, "wssb_Lung_test_psnr_gt_e" : 30.61, 
    "wssb_Breast_test_psnr_gt_h" : 32.20, "wssb_Breast_test_psnr_gt_e" : 29.43, 
    "wssb_Colon_test_psnr_gt_h" : 34.08, "wssb_Colon_test_psnr_gt_e" : 33.32}, ignore_index=True)

df["wssb_mean_test_psnr_gt_h"] = df[[c for c in df.columns if "h" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt_e"] = df[[c for c in df.columns if "e" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt"] = df[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]].mean(axis=1)

df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

<ipython-input-57-86acceeb6b2d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name" : "BKSVD",
<ipython-input-57-86acceeb6b2d>:19: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df["wssb_mean_test_psnr_gt_e"] = df[[c for c in df.columns if "e" in c]].mean(axis=1)


BCDnet $\theta=0.0$ BCDnet $\theta=0.5$ BCDnet $\theta=0.75$  \
Organ  Stain                                                                
Lung   H               23.202609           25.936085            18.568592   
       E                6.120045           13.685157           -10.185472   
Breast H               17.554499            22.38586            13.433162   
       E               10.417577           22.532263           -21.936788   
Colon  H               18.063859           22.295033             15.18085   
       E               11.617042           18.868812           -13.768965   
Mean   H               19.606989           23.538993            15.727535   
       E               15.226089           21.320315             2.431273   
       Mean            17.416539           22.429654             9.079404   

             BCDnet $\theta=0.9$ BCDnet $\theta=0.99$      BKSVD  
Organ  Stain                                                      
Lung   H               -6.717244           -14.863313      32.67  
       E                -7.28731           -12.696298      30.61  
Breast H              -17.660843           -21.584594       32.2  
       E              -12.224435           -12.419947      29.43  
Colon  H              -15.820709           -19.474808      34.08  
       E               -6.197699            -7.809068      33.32  
Mean   H              -13.399599           -18.640905  32.983333  
       E              -11.329691           -15.355562  32.184762  
       Mean           -12.364645           -16.998233  32.584048

In [58]:
styler = df.style
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Ground truth PSNR on WSSB dataset.", label="tab:gt_psnr_wssb",
    )
print(s)

\begin{table}
\centering
\caption{Ground truth PSNR on WSSB dataset.}
\label{tab:gt_psnr_wssb}
\begin{tabular}{cccccccc}
\toprule
 &  & BCDnet $	heta=0.0$ & BCDnet $	heta=0.5$ & BCDnet $	heta=0.75$ & BCDnet $	heta=0.9$ & BCDnet $	heta=0.99$ & BKSVD \\
Organ & Stain &  &  &  &  &  &  \\
\midrule
\multirow[c]{2}{*}{Lung} & H & 23.202609 & 25.936085 & 18.568592 & -6.717244 & -14.863313 & 32.670000 \\
 & E & 6.120045 & 13.685157 & -10.185472 & -7.287310 & -12.696298 & 30.610000 \\
\cline{1-8}
\multirow[c]{2}{*}{Breast} & H & 17.554499 & 22.385860 & 13.433162 & -17.660843 & -21.584594 & 32.200000 \\
 & E & 10.417577 & 22.532263 & -21.936788 & -12.224435 & -12.419947 & 29.430000 \\
\cline{1-8}
\multirow[c]{2}{*}{Colon} & H & 18.063859 & 22.295033 & 15.180850 & -15.820709 & -19.474808 & 34.080000 \\
 & E & 11.617042 & 18.868812 & -13.768965 & -6.197699 & -7.809068 & 33.320000 \\
\cline{1-8}
\multirow[c]{3}{*}{Mean} & H & 19.606989 & 23.538993 & 15.727535 & -13.399599 & -18.640905 & 32.983333 

In [59]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
theta_list = list(df["theta_val"])
df = df.drop(columns=[c for c in df.columns if "loss" in c])
df = df[df["pretraining_epochs"] == 0] 
df = df[[c for c in df.columns if "ssim_gt" in c]]
df["Name"] = [f"BCDnet $\\theta={t}$" for t in theta_list]
df = df[["Name"] + list(df.columns[:-1])]
#df = df.append({"Name" : "BCDnet (old)", 
#    "wssb_Lung_test_ssim_gt_h" : 0.9518, "wssb_Lung_test_ssim_gt_e" : 0.8201, 
#    "wssb_Breast_test_ssim_gt_h" : 0.8962, "wssb_Breast_test_ssim_gt_e" : 0.7321, 
#    "wssb_Colon_test_ssim_gt_h" : 0.8969, "wssb_Colon_test_ssim_gt_e" : 0.8560}, ignore_index=True)
df = df.append({"Name" : "BKSVD", 
    "wssb_Lung_test_ssim_gt_h" : 0.9764, "wssb_Lung_test_ssim_gt_e" : 0.9461, 
    "wssb_Breast_test_ssim_gt_h" : 0.9801, "wssb_Breast_test_ssim_gt_e" : 0.9632, 
    "wssb_Colon_test_ssim_gt_h" : 0.9826, "wssb_Colon_test_ssim_gt_e" : 0.9646}, ignore_index=True)
df["wssb_mean_test_ssim_gt_h"] = df[[c for c in df.columns if "h" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt_e"] = df[[c for c in df.columns if "e" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt"] = df[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]].mean(axis=1)

df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

<ipython-input-59-fec6f0547a40>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name" : "BKSVD",
<ipython-input-59-fec6f0547a40>:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df["wssb_mean_test_ssim_gt_e"] = df[[c for c in df.columns if "e" in c]].mean(axis=1)


BCDnet $\theta=0.0$ BCDnet $\theta=0.5$ BCDnet $\theta=0.75$  \
Organ  Stain                                                                
Lung   H                 0.80463            0.953625             0.860915   
       E                 0.99127            0.990705             0.992205   
Breast H                0.683078            0.889683             0.704519   
       E                0.923056            0.949896             0.891647   
Colon  H                0.686379            0.923319             0.734011   
       E                0.867262            0.872885             0.870677   
Mean   H                0.724696            0.922209             0.766481   
       E                0.811482            0.928903             0.831493   
       Mean             0.768089            0.925556             0.798987   

             BCDnet $\theta=0.9$ BCDnet $\theta=0.99$     BKSVD  
Organ  Stain                                                     
Lung   H                0.425555             0.425916    0.9764  
       E                 0.99221             0.992217    0.9461  
Breast H                0.375958             0.376701    0.9801  
       E                0.891662             0.891657    0.9632  
Colon  H                0.438439             0.438478    0.9826  
       E                0.870756             0.870736    0.9646  
Mean   H                0.413317             0.413698    0.9797  
       E                  0.6297             0.629915  0.970386  
       Mean             0.521508             0.521807  0.975043

In [60]:
styler = df.style
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Ground truth SSIM on WSSB dataset.", label="tab:gt_ssim_wssb",
    )
print(s)

\begin{table}
\centering
\caption{Ground truth SSIM on WSSB dataset.}
\label{tab:gt_ssim_wssb}
\begin{tabular}{cccccccc}
\toprule
 &  & BCDnet $	heta=0.0$ & BCDnet $	heta=0.5$ & BCDnet $	heta=0.75$ & BCDnet $	heta=0.9$ & BCDnet $	heta=0.99$ & BKSVD \\
Organ & Stain &  &  &  &  &  &  \\
\midrule
\multirow[c]{2}{*}{Lung} & H & 0.804630 & 0.953625 & 0.860915 & 0.425555 & 0.425916 & 0.976400 \\
 & E & 0.991270 & 0.990705 & 0.992205 & 0.992210 & 0.992217 & 0.946100 \\
\cline{1-8}
\multirow[c]{2}{*}{Breast} & H & 0.683078 & 0.889683 & 0.704519 & 0.375958 & 0.376701 & 0.980100 \\
 & E & 0.923056 & 0.949896 & 0.891647 & 0.891662 & 0.891657 & 0.963200 \\
\cline{1-8}
\multirow[c]{2}{*}{Colon} & H & 0.686379 & 0.923319 & 0.734011 & 0.438439 & 0.438478 & 0.982600 \\
 & E & 0.867262 & 0.872885 & 0.870677 & 0.870756 & 0.870736 & 0.964600 \\
\cline{1-8}
\multirow[c]{3}{*}{Mean} & H & 0.724696 & 0.922209 & 0.766481 & 0.413317 & 0.413698 & 0.979700 \\
 & E & 0.811482 & 0.928903 & 0.831493 & 0.629700 & 